In [2]:
import pandas as pd

In [1]:
print(1)

1


In [3]:
pd.merge?

Signature:
pd.merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'str' = 'inner',
    on: 'IndexLabel | None' = None,
    left_on: 'IndexLabel | None' = None,
    right_on: 'IndexLabel | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool' = True,
    indicator: 'bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

.. warning::

    If both key columns contain rows where the key is a null

In [5]:
referendum = pd.read_csv("./data/referendum.csv", sep=';')
regions = pd.read_csv("./data/regions.csv")
departments = pd.read_csv("./data/departments.csv")

In [12]:
rd_merge = pd.merge(
        regions[['code', 'name']],
        departments[['region_code','code','name']],

        left_on='code',
        right_on='region_code',

        how='left',

        suffixes=('_reg', '_dep')
    ).drop(columns=['region_code'])


In [13]:
rd_merge

,code_reg,name_reg,code_dep,name_dep
0,01,Guadeloupe,971,Guadeloupe
1,02,Martinique,972,Martinique
2,03,Guyane,973,Guyane
3,04,La Réunion,974,La Réunion
4,06,Mayotte,976,Mayotte
...,...,...,...,...
104,COM,Collectivités d'Outre-Mer,984,Terres australes et antarctiques françaises
105,COM,Collectivités d'Outre-Mer,986,Wallis et Futuna
106,COM,Collectivités d'Outre-Mer,987,Polynésie française
107,COM,Collectivités d'Outre-Mer,988,Nouvelle-Calédonie
